**Analyzing the Immune Epitope Database (IEDB)**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/epitope-prediction')  

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/input_covid.csv
./input/input_bcell.csv
./input/input_sars.csv


In [3]:
import pandas as pd 

# Importing the dataset
bcell = pd.read_csv('./input/input_bcell.csv')

# Only the b cell dataset is being used on this notebook for training and the covid dataset for testing (the sars dataset is not used for training)
bcell.head()

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
0,A2T3T0,MDVLYSLSKTLKDARDKIVEGTLYSNVSDLIQQFNQMIITMNGNEF...,161,165,SASFT,1.016,0.703,1.018,2.22,5.810364,0.103275,-0.143829,40.273300,1
1,F0V2I4,MTIHKVAINGFGRIGRLLFRNLLSSQGVQVVAVNDVVDIKVLTHLL...,251,255,LCLKI,0.770,0.179,1.199,-3.86,6.210876,0.065476,-0.036905,24.998512,1
2,O75508,MVATCLQVVGFVTSFVGWIGVIVTTSTNDWVVTCGYTIPTCRKLDE...,145,149,AHRET,0.852,3.427,0.960,4.28,8.223938,0.091787,0.879227,27.863333,1
3,O84462,MTNSISGYQPTVTTSTSSTTSASGASGSLGASSVSTTANATVTQTA...,152,156,SNYDD,1.410,2.548,0.936,6.32,4.237976,0.044776,-0.521393,30.765373,1
4,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,85,89,DGTYR,1.214,1.908,0.937,4.64,6.867493,0.103846,-0.578846,21.684615,1


In [4]:
bcell.target.value_counts()

0    10485
1     3902
Name: target, dtype: int64

In [5]:
# Splitting using train_test_split form sklearn.model_selection

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

bcell = shuffle(bcell)

X = bcell.drop(['target'], axis = 1)
y = bcell['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Transforming traning dataset
X_train = X_train.drop(['parent_protein_id', 'protein_seq', 'peptide_seq'], axis = 1)
X_train["length"] = X_train["end_position"] - X_train["start_position"] + 1


# Transforming testing dataset
X_test = X_test.drop(['parent_protein_id', 'protein_seq', 'peptide_seq'], axis = 1)
X_test["length"] = X_test["end_position"] - X_test["start_position"] + 1

                                              

In [6]:
# Splitting the dataset manually

train_pct_index = int(0.8 * len(X))
X_train, X_test = X[:train_pct_index], X[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]


# Transforming traning dataset
X_train = X_train.drop(['parent_protein_id', 'protein_seq', 'peptide_seq'], axis = 1)
X_train["length"] = X_train["end_position"] - X_train["start_position"] + 1


# Transforming testing dataset
X_test = X_test.drop(['parent_protein_id', 'protein_seq', 'peptide_seq'], axis = 1)
X_test["length"] = X_test["end_position"] - X_test["start_position"] + 1


In [ ]:
!pip install yellowbrick==0.9.1 scikit-learn==0.22.2
!pip install imbalanced-learn

In [ ]:

!pip install mlrose
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install imblearn


In [9]:
#Defining the pipeline

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler


pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=10)

In [10]:

# Evaluating the model using cross-validation without tune hyperparameters (training dataset)

from sklearn.model_selection import cross_val_score
import numpy as np

f1 = cross_val_score(pipeline, X_train, y_train, scoring='f1', cv=stratified_kfold, n_jobs=-1)
roc_auc = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=stratified_kfold, n_jobs=-1)

# report performance
print('F1:', np.mean(f1))
print('AUC:', np.mean(roc_auc))

F1: 0.782051948304899
AUC: 0.9358319844782084


In [11]:
# Evaluating the model using cross-validation and tune hyperparameters (training dataset)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")


param_grid = {"classifier__max_depth": [3, None],
              "classifier__max_features": [1, 3, 10],
              "classifier__min_samples_split": [1, 3, 10],
              "classifier__min_samples_leaf": [1, 3, 10],
              "classifier__criterion": ["gini", "entropy"]}


# Creating a dictionary with the metrics to be calculated
scores = {'accuracy' :make_scorer(accuracy_score),
               'recall'   :make_scorer(recall_score),
               'precision':make_scorer(precision_score),
               'f1'       :make_scorer(fbeta_score, beta = 1),
               'roc_auc'  :make_scorer(roc_auc_score) }


grid_search = GridSearchCV(estimator = pipeline,
                      param_grid = param_grid,
                      cv = stratified_kfold,
                      scoring = scores,   
                      refit = 'f1'
                      )           


grid_search.fit(X_train, y_train)

# Prints the column names that are stored in grid_search.
#pd.DataFrame(grid_search.cv_results_).columns.tolist()


result = pd.DataFrame(grid_search.cv_results_)[[
                                  'mean_test_recall',
                                  'mean_test_precision',
                                  'mean_test_f1',
                                  'mean_test_roc_auc'
                                  ]]

print(result.sort_values(ascending=False, by='mean_test_f1').head())




     mean_test_recall  mean_test_precision  mean_test_f1  mean_test_roc_auc
91           0.790137             0.780605      0.785084           0.853557
37           0.786937             0.773992      0.780156           0.850584
104          0.789830             0.769096      0.779152           0.850659
100          0.786622             0.772130      0.779043           0.849950
46           0.787585             0.769436      0.778235           0.849715


In [12]:

# Evaluating test set prediction without hyperparameter tuning
pipeline.fit(X_train, y_train)

pred = pipeline.predict(X_test)
print("F1 : ",fbeta_score(y_test, pred, beta=0.5))
print("AUC: ", roc_auc_score(y_test,pred))


F1 :  0.7798726738491676
AUC:  0.8652430919987837


In [13]:
# Evaluating test set prediction with hyperparameter tuning
pred=grid_search.predict(X_test)
print("F1 : ",fbeta_score(y_test, pred, beta=0.5))
print("AUC: ", roc_auc_score(y_test,pred))

F1 :  0.7823129251700679
AUC:  0.8695155325806547


As we can see above, tuning the hyperparameters slightly improves the model's performance.